In [1]:
import time
from pathlib import Path

from pyodm import Node
from tqdm.notebook import tqdm

# NodeODM

We have [NodeODM](https://github.com/OpenDroneMap/NodeODM) running as a service on `http://nodeodm`. This service is currently only reachable from the Hub.

## 1. Connect to NodeODM

### 1.1. Bash

In [2]:
%%bash
curl http://nodeodm/info

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   187  100   187    0     0   2595      0 --:--:-- --:--:-- --:--:--  2633


{"version":"2.2.1","taskQueueCount":0,"totalMemory":809774592000,"availableMemory":630647992320,"cpuCores":80,"maxImages":null,"maxParallelTasks":2,"engineVersion":"3.0.3","engine":"odm"}

### 1.2. PyODM

We can use [PyODM](https://github.com/OpenDroneMap/PyODM) to interact with the API.

In [3]:
node = Node.from_url("http://nodeodm")
print(node.info())

{'version': '2.2.1', 'task_queue_count': 0, 'total_memory': 809774592000, 'available_memory': 630643826688, 'cpu_cores': 80, 'max_images': None, 'max_parallel_tasks': 2, 'engine': 'odm', 'engine_version': '3.0.3', 'odm_version': '?'}


## 2. Orthomosaic images

In [4]:
# Path to raw images
# image_fold = r"/home/notebook/shared-seabee-ns9879k/seabirds/2022/Bergen_KalandsvannetGrasskjeret_20220525/images"
# image_fold = r"/home/notebook/shared-seabee-ns9879k/seabirds/2022/Oslo_NorskGjenvinning_20220619/images"
image_fold = r"/home/notebook/shared-seabee-ns9879k/seabirds/2022/Fedje_Fedjemyran_20220920/images"

In [5]:
# Get image paths
image_fold = Path(image_fold)
image_files = list(image_fold.glob("*.JPG"))
images_size = sum(f.stat().st_size for f in image_files) / 1e9
print(f"{len(image_files)} files to process with a total size of {images_size:.2f} GB.")
image_files = [str(i) for i in image_files]

1783 files to process with a total size of 15.30 GB.


In [6]:
# Send task to NodeODM. Options are documented here:
# https://docs.opendronemap.org/arguments/
nodeodm_options = {
    "dsm": True,
    "dtm": True,
    "cog": True,
    "orthophoto-compression": "LZW",
    "orthophoto-resolution": 1,  # cm/pixel. If set very small, output will be auto-limited by data to max sensible value
    "dem-resolution": 1,  # cm/pixel. If set very small, output will be auto-limited by data to max sensible value
    "max-concurrency": 16,
    "auto-boundary": True,
    "feature-quality": "medium",  # ultra | high | medium | low | lowest
    "pc-quality": "low",  # ultra | high | medium | low | lowest
    "fast-orthophoto": False,
    "split": 100,
    # "split-overlap": 50,
}
task = node.create_task(
    image_files,
    nodeodm_options,
)

We can wait for the task to finish with `task.wait_for_completion()` (which blocks), but better to let it run in the background if you want to do other stuff. Alternatively, the code below provides a progress bar (but this also blocks).

In [ ]:
with tqdm(total=100) as pbar:
    while (task.info().progress < 100) and (task.info().last_error == ""):
        time.sleep(5)
        cur_perc = int(task.info().progress)
        pbar.update(cur_perc - pbar.n)
    if task.info().last_error == "":
        print(
            f"Completed successfully in {task.info().processing_time/60000:.1f} minutes."
        )
    else:
        print("Failed with error:")
        print(task.info().last_error)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# # Some other useful commands

# Get a task using uid
# task = node.get_task('uid')

# # Get task info
# print(task.info())

# # Cancel a task
# task.cancel()

# # Download results to subfolder 'results' in the notebook directory
# task.download_assets("./results")

# # The task files are also available as a read-only folder
# print(f"/home/notebook/shared-seabee-ns9879k/nodeodm-workdir/{task.info().uuid}")